In [1]:
import numpy as np
from spectral_cube import SpectralCube
from astropy.io import fits
from reproject import reproject_interp
from tqdm.auto import tqdm
import pyspeckit
from astropy.table import Table, hstack, join
from astropy import stats
import os
import astropy.units as u
import astropy.constants as c

In [61]:
def average_spectrum_for_indexes(hdu_regrid, index_map_hdu):
    
    index_map = index_map_hdu.data.copy()
    index_map[index_map==0] = np.nan
    index_map_is_dust = np.ones(index_map.shape) *np.nan
    
    # Load the data cube
    unique_indexes = np.unique(index_map)
    
    stats_id = np.ones(len(unique_indexes)) *np.nan
    stats_min = np.ones(len(unique_indexes)) *np.nan
    stats_mean = np.ones(len(unique_indexes)) *np.nan
    stats_max = np.ones(len(unique_indexes)) *np.nan
    stats_rms = np.ones(len(unique_indexes)) *np.nan 
    is_dust = np.array([False]*len(unique_indexes))
    
    i = 0
    for index in tqdm(unique_indexes):
        
        if i > 1: 
            continue
        
        # Create a mask for the current index
        mask = (index_map == index)

        data = hdu_regrid.data.copy()
        masked_data = data[mask]
        
        stats_id[i] = index
        stats_min[i] = np.nanmin(masked_data)
        stats_mean[i] = np.nanmean(masked_data)
        stats_max[i] = np.nanmax(masked_data)
        stats_rms[i] = stats.mad_std(data)
        is_dust[i] = stats_mean[i]<(stats_rms[i]*3)
        
        # Add ID to new index array
        if is_dust[i] == True: 
            index_map_is_dust[mask] = index
        
        i+=1
    
    return (stats_id, stats_min, stats_mean, stats_max, stats_rms, is_dust), index_map_is_dust

In [46]:
# Define files 
index_map_filename = "/Users/abarnes/Dropbox/work/scratch/ACES1/data/12m_cont/ACES_leaf_mask_3_1_mp179_pb0.55_pp6_pm2_ar3.0_sm25.fits"
catalog_filename = '/Users/abarnes/Dropbox/work/scratch/ACES1/data/12m_cont/aces_catalog_3_1_mp179_pb0.55_pp6_pm2_ar3.0_sm25.fits'

meerkat_filename = '/Users/abarnes/Dropbox/work/Smallprojects/ppVII/review/cover/data/meerkat.fits'

In [47]:
#Load catalog 
catalog = Table.read(catalog_filename)

In [48]:
#Open index map 
index_map_hdu = fits.open(index_map_filename)[0]

#Open Meerkat image
meerkat_hdu = fits.open(meerkat_filename)[0]

In [49]:
if not os.path.exists('./meerkat_hdu_regrid.fits'):
    print('[INFO] Making Meerkat regrid...')
    meerkat_data_regrid, _ = reproject_interp(meerkat_hdu, index_map_hdu.header, parallel=True)
    meerkat_hdu_regrid = fits.PrimaryHDU(meerkat_data_regrid, index_map_hdu.header)
    meerkat_hdu_regrid.writeto('./meerkat_hdu_regrid.fits')
else:
    print('[INFO] Loading Meerkat regrid...')
    meerkat_hdu_regrid = fits.open('./meerkat_hdu_regrid.fits')[0]

[INFO] Loading Meerkat regrid...


In [62]:
#Get Meerkat stats
stats_out, index_map_is_dust = average_spectrum_for_indexes(meerkat_hdu_regrid, index_map_hdu)

  0%|          | 0/690 [00:00<?, ?it/s]

True
now
True
now


In [63]:
#Make table
catalog_meerkat = Table(stats_out, names=['index', 'meerkat_stats_min', 'meerkat_stats_mean', 'meerkat_stats_max', 'meerkat_stats_rms', 'meerkat_is_dust'])
catalog_all = join(catalog, catalog_meerkat)

#Save catalog
catalog_all.write(catalog_filename.replace('.fits', '_meerkat.fits'), overwrite=True)

#Save index map
index_map_is_dust_hdu = fits.PrimaryHDU(index_map_is_dust, index_map_hdu.header)
index_map_is_dust_hdu.writeto(index_map_filename.replace('.fits','_meerkat.fits'), overwrite=True)

In [64]:
catalog_meerkat

index,meerkat_stats_min,meerkat_stats_mean,meerkat_stats_max,meerkat_stats_rms,meerkat_is_dust
float64,float64,float64,float64,float64,bool
27.0,8.434355051040144e-07,6.409451905871027e-06,9.176300055921512e-06,1.6832057695951516e-05,True
31.0,-5.738839456853758e-06,3.63014739620887e-06,1.1898472683025845e-05,1.6832057695951516e-05,True
nan,nan,nan,nan,nan,False
nan,nan,nan,nan,nan,False
nan,nan,nan,nan,nan,False
nan,nan,nan,nan,nan,False
nan,nan,nan,nan,nan,False
nan,nan,nan,nan,nan,False
nan,nan,nan,nan,nan,False
